In [1]:
# from tensorflow.keras.layers import (
#     multiply,
#     concatenate,
# )
# from tensorflow.keras.layers import GaussianNoise, GaussianDropout
# from tensorflow.keras.layers import Lambda

# from tensorflow.keras.utils import to_categorical

# import tensorflow.keras.backend as K

In [2]:
# import pickle
# import cv2

# import PIL

# from IPython import display

In [3]:
import tensorflow as tf

# https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py
# from keras.datasets import mnist
from tensorflow.keras.layers import (
    Input,
    Dense,
    Reshape,
    Flatten,
    Dropout,
    BatchNormalization,
    Activation,
    ZeroPadding2D,
    LeakyReLU,
    UpSampling2D,
    Conv2D,
    MaxPooling2D,
    Concatenate,
)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import h5py
import pathlib

import time

import math

In [4]:
print("Tensorflow version: ", tf.VERSION)  # 1.15.2
print("Keras version: ", tf.keras.__version__)  # 2.2.4-tf

tf.enable_eager_execution()
print("Is eager execution enabled: ", tf.executing_eagerly())
print("Is there a GPU available: ", tf.test.is_gpu_available())

Tensorflow version:  1.15.2
Keras version:  2.2.4-tf
Is eager execution enabled:  True
Is there a GPU available:  True


In [5]:
path_full_tfrecord = "/data/fp85.tfrecord"
dir_save = "rooms"

In [6]:
# https://medium.com/@moritzkrger/speeding-up-keras-with-tfrecord-datasets-5464f9836c36

BUFFER_SIZE = 1024


def _parse_function(example_proto):
    # Create a description of the features.
    feature_description = {
        "floorplan": tf.io.FixedLenFeature(
            [28, 28, 6], tf.float32, default_value=tf.zeros([28, 28, 6], tf.float32)
        ),
        "plan_id": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "norm_year": tf.io.FixedLenFeature([], tf.float32, default_value=-1.0),
        "sido": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "norm_area": tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
        "num_rooms": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "num_baths": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "brands": tf.io.FixedLenFeature(
            [12], tf.int64, default_value=tf.zeros([12], tf.int64)
        ),
    }

    # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)

    return (parsed_example["floorplan"], parsed_example["num_rooms"])


def _onehot_fp(fp, *rest):
    """
    Replace unit layer with outdoor layer and restrict function space layers indoor
    """
    fp_func = fp[:, :, 1:6] * tf.reshape(fp[:, :, 0], (28, 28, 1))
    fp_out = tf.reshape(1 - fp[:, :, 0], (28, 28, 1))
    return (tf.concat([fp_out, fp_func], axis=2), *rest)


def _rescale_fp(fp, *rest):
    """
    Rescale from [0, 1] to [-1, 1]
    """
    return (fp * 2 - 1, *rest)


def _onehot_sido(_, sido, *rest):
    """
    Make sido label one-hot array
    """
    n_classes = 9
    return (_, tf.one_hot(sido, n_classes), *rest)


def _onehot_bath(_, n_rms, *rest):
    """
    Make sido label one-hot array
    """
    n_classes = 2
    return (_, tf.one_hot(n_rms - 1, n_classes), *rest)


def _visualize_fp_onehot(fps):
    # adjusted for different luminance
    channel_to_rgba = np.array(
        [
            [0.0, 0.0, 0.0, 0.0],  # ignore outdoor mask
            [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
            [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
            [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
            [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
            [0.0, 0.81, 0.76, 0.0],
        ]
    )  # bathroom to cyan L75

    # make colors subtractive
    channel_to_rgba[1:6, 0:3] -= 1

    # put it on white
    fps_rgba = np.clip(
        np.array([1.0, 1.0, 1.0, 1.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
    )
    return fps_rgba


def create_dataset(filepath, batch_size=8):

    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepath, compression_type="GZIP")

    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=4)

    # make the floorplan one-hot-ish
    dataset = dataset.map(_onehot_fp, num_parallel_calls=4)

    # rescale to [-1, 1]
    dataset = dataset.map(_rescale_fp, num_parallel_calls=4)

    # sido one-hot
    dataset = dataset.map(_onehot_bath, num_parallel_calls=4)

    # This dataset will go on forever
    dataset = dataset.repeat()

    # Set the number of datapoints you want to load and shuffle
    dataset = dataset.shuffle(BUFFER_SIZE)

    # Set the batchsize
    dataset = dataset.batch(batch_size)

    return dataset

In [7]:
# dataset = create_dataset(path_full_tfrecord)
# dataset_iter = tf.compat.v1.data.make_one_shot_iterator(dataset)
# plt.imshow(_visualize_fp_onehot(next(dataset_iter)[0][0]))

# CDCGAN

참고:
- https://github.com/gaborvecsei/CDCGAN-Keras/
- https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/

In [8]:
class CDCGAN_fp_sido:
    def __init__(self, dir_save="cdcgan"):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 6
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        self.n_classes = 2

        self.latent_dim = 100
        self.dir_save = dir_save
        self.model_name = "cdcgan"

        pathlib.Path(self.dir_save).mkdir(parents=True, exist_ok=True)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]
        )

        # Build and compile the combined model
        self.generator = self.build_generator()

        self.combined = self.build_combined(self.generator, self.discriminator)
        self.combined.compile(loss="binary_crossentropy", optimizer=optimizer)

    def build_generator(self):
        # Prepare noise input
        input_z = Input(shape=(self.latent_dim,))
        dense_z_1 = Dense(1024)(input_z)
        act_z_1 = Activation("tanh")(dense_z_1)

        dense_z_2 = Dense(128 * 7 * 7)(act_z_1)
        bn_z_1 = BatchNormalization()(dense_z_2)
        reshape_z = Reshape((7, 7, 128), input_shape=(128 * 7 * 7,))(bn_z_1)

        # Prepare Conditional (label) input
        input_c = Input(shape=(self.n_classes,))
        dense_c_1 = Dense(1024)(input_c)
        act_c_1 = Activation("tanh")(dense_c_1)
        dense_c_2 = Dense(128 * 7 * 7)(act_c_1)
        bn_c_1 = BatchNormalization()(dense_c_2)
        reshape_c = Reshape((7, 7, 128), input_shape=(128 * 7 * 7,))(bn_c_1)

        # Combine input source
        concat_z_c = Concatenate()([reshape_z, reshape_c])

        # Image generation with the concatenated inputs
        up_1 = UpSampling2D(size=(2, 2))(concat_z_c)
        conv_1 = Conv2D(64, (5, 5), padding="same")(up_1)
        act_1 = Activation("tanh")(conv_1)
        up_2 = UpSampling2D(size=(2, 2))(act_1)
        conv_2 = Conv2D(self.channels, (5, 5), padding="same")(up_2)
        act_2 = Activation("tanh")(conv_2)

        model = Model(inputs=[input_z, input_c], outputs=act_2, name="generator",)
        model.summary()
        return model

    def build_discriminator(self):
        # generated image input
        input_gen_image = Input(shape=self.img_shape)
        conv_1_image = Conv2D(64, (5, 5), padding="same")(input_gen_image)
        act_1_image = Activation("tanh")(conv_1_image)
        pool_1_image = MaxPooling2D(pool_size=(2, 2))(act_1_image)
        conv_2_image = Conv2D(128, (5, 5))(pool_1_image)
        act_2_image = Activation("tanh")(conv_2_image)
        pool_2_image = MaxPooling2D(pool_size=(2, 2))(act_2_image)

        # label input
        input_c = Input(shape=(self.n_classes,))
        dense_1_c = Dense(1024)(input_c)
        act_1_c = Activation("tanh")(dense_1_c)
        dense_2_c = Dense(5 * 5 * 128)(act_1_c)
        bn_c = BatchNormalization()(dense_2_c)
        reshaped_c = Reshape((5, 5, 128))(bn_c)

        # combine input source
        concat = Concatenate()([pool_2_image, reshaped_c])

        # real or fake 0-1 output
        flat = Flatten()(concat)
        dense_1 = Dense(1024)(flat)
        act_1 = Activation("tanh")(dense_1)
        dense_2 = Dense(1)(act_1)
        act_2 = Activation("sigmoid")(dense_2)

        model = Model(
            inputs=[input_gen_image, input_c], outputs=act_2, name="discriminator",
        )
        model.summary()
        return model

    def build_combined(self, g, d):
        # The generator takes noise and class as input and generates imgs
        input_z = Input(shape=(self.latent_dim,))
        input_c = Input(shape=(self.n_classes,))
        gen_image = g([input_z, input_c])

        # For the combined model we will only train the generator
        d.trainable = False

        # The discriminator takes generated images and classes as input and determines validity
        valid = d([gen_image, input_c])

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        model = Model(inputs=[input_z, input_c], outputs=valid, name="combined")
        model.summary()
        return model

    def train(
        self,
        path_tfrecord,
        epochs=201,
        initial_epoch=0,
        batch_size=128,
        save_interval=1,
    ):
        if initial_epoch:
            self.load_model(initial_epoch - 1)

        n_examples = sum(
            1 for _ in tf.data.TFRecordDataset(path_tfrecord, compression_type="GZIP")
        )
        batches_per_epoch = math.ceil(n_examples / batch_size)

        # create a floorplan data iterator with size of half batch
        train_dataset = create_dataset(path_tfrecord, batch_size=batch_size // 2)
        dataset_iter = tf.compat.v1.data.make_one_shot_iterator(train_dataset)

        # Adversarial ground truths
        valid_full = np.ones((batch_size, 1))
        valid_half = np.ones((batch_size // 2, 1))
        fake_half = np.zeros((batch_size // 2, 1))

        # zero knowlegde guess for starting loss
        d_loss_fake = [-math.log(0.5), 0.5]
        g_loss = -math.log(0.5)

        # start time

        t0 = time.time()
        t1 = t0

        for epoch in range(initial_epoch, initial_epoch + epochs):
            for _ in range(batches_per_epoch):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Load a half batch of floorplan images and labels
                batch = next(dataset_iter)
                img, label_half = batch[0].numpy(), batch[1].numpy()

                # Sample noise and generate a half batch of new images
                noise_half = np.random.normal(0, 1, (batch_size // 2, self.latent_dim))

                gen_img = self.generator.predict([noise_half, label_half])

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(
                    [img, label_half], valid_half
                )
                d_loss_fake = self.discriminator.train_on_batch(
                    [gen_img, label_half], fake_half
                )
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Sample noise and generate a batch of new images
                noise_full = np.random.normal(0, 1, (batch_size, self.latent_dim))
                label_full = np.tile(label_half, (2, 1))

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(
                    [noise_full, label_full], valid_full
                )

            # If at save interval => save generated image samples
            if epoch % save_interval == 0 or epoch == initial_epoch + epochs - 1:

                # Plot the progress
                print(
                    "%d [D loss: %f, D_fake acc.: %.1f%%] [G loss: %f]"
                    % (epoch, d_loss[0], 100 * d_loss_fake[1], g_loss),
                    end=" ",
                )

                self.save_imgs(epoch)

                t2 = time.time()
                print(f"{t2-t1:.1f}s elapsed since, total {t2-t0:.1f}s")
                t1 = t2

        # save model after finish
        self.save_model(epoch)

    def _visualize_fp(self, fps):
        # adjusted for different luminance
        channel_to_rgba = np.array(
            [
                [0.0, 0.0, 0.0, 1.0],  # unit mask to alpha
                [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
                [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
                [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
                [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
                [0.0, 0.81, 0.76, 0.0],
            ]
        )  # bathroom to cyan L75

        # make colors subtractive
        channel_to_rgba[1:6, 0:3] -= 1

        # put it on transparent white
        fps_rgba = np.clip(
            np.array([1.0, 1.0, 1.0, 0.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
        )
        return fps_rgba

    def _visualize_fp_onehot(self, fps):
        # adjusted for different luminance
        channel_to_rgba = np.array(
            [
                [0.0, 0.0, 0.0, 0.0],  # ignore outdoor mask
                [0.0, 0.33, 0.0, 0.0],  # entrance to green L30
                [1.0, 0.25, 0.0, 0.0],  # LDK to red L57
                [0.0, 0.26, 1.0, 0.0],  # bedroom to blue L40
                [0.83, 0.87, 0.0, 0.0],  # balcony to yellow L85
                [0.0, 0.81, 0.76, 0.0],
            ]
        )  # bathroom to cyan L75

        # make colors subtractive
        channel_to_rgba[1:6, 0:3] -= 1

        # put it on white
        fps_rgba = np.clip(
            np.array([1.0, 1.0, 1.0, 1.0]) + (np.array(fps) @ channel_to_rgba), 0, 1
        )
        return fps_rgba

    def save_imgs(self, epoch, r=5, c=-1):
        if c == -1:
            c = self.n_classes
        ### create a fixed noise
        # save the current state
        random_state = np.random.get_state()
        # set same noise for images
        np.random.seed(1106)
        noise = np.repeat(np.random.normal(0, 1, (r, self.latent_dim)), c, axis=0)
        # reset the state
        np.random.set_state(random_state)

        label = np.tile(np.eye(c), (r, 1))

        gen_imgs = self.generator.predict([noise, label])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        # visualize as rgba
        gen_imgs_rgba = self._visualize_fp_onehot(gen_imgs)

        fig, axs = plt.subplots(r, c, figsize=(c / 2.5, r / 2.5), dpi=300)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs_rgba[cnt, :, :, :])
                axs[i, j].axis("off")
                cnt += 1
        fig.savefig(
            f"{self.dir_save}/{self.model_name}_{epoch:06}.png",
            bbox_inches="tight",
            pad_inches=0,
        )
        plt.close()

    def save_model(self, end_epoch):
        for path, model in zip(
            [
                f"{self.dir_save}/{self.model_name}_{end_epoch:06}_{name}.h5"
                for name in ["gen", "disc"]
            ],
            [self.generator, self.discriminator],
        ):
            with h5py.File(path, "w") as file:
                weight = model.get_weights()
                for i in range(len(weight)):
                    file.create_dataset("weight" + str(i), data=weight[i])

    def load_model(self, end_epoch):
        for path, model in zip(
            [
                f"{self.dir_save}/{self.model_name}_{end_epoch:06}_{name}.h5"
                for name in ["gen", "disc"]
            ],
            [self.generator, self.discriminator],
        ):
            with h5py.File(path, "r") as file:
                weight = []
                for i in range(len(file.keys())):
                    weight.append(file["weight" + str(i)][:])

            model.set_weights(weight)

In [9]:
model = CDCGAN_fp_sido(dir_save)

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 6)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   9664        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 64)   0           conv2d[0][0]                     
______________________________________________________________________________________

Model: "combined"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
generator (Model)               (None, 28, 28, 6)    13433542    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
discriminator (Model)           (None, 1)            10066113    generator[1][0]           

In [ ]:
model.train(
    path_full_tfrecord,
    epochs=1000,
    #     initial_epoch=1000,
    batch_size=128,
    save_interval=10,
)

# 128*1000 -> less than 202s (now skip training on overpowering one)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
0 [D loss: 0.385995, D_fake acc.: 76.6%] [G loss: 2.730809] 27.4s elapsed since, total 27.4s
10 [D loss: 0.002596, D_fake acc.: 100.0%] [G loss: 0.051477] 253.5s elapsed since, total 280.9s
